# Analyzing CIA Factbook Data Using SQL

In this project, we'll work with data from the CIA World Factbook, a compendium of statistics about all of the countries on Earth. 

The Factbook contains demographic information like:

- `population` - The population as of 2015.
- `population_growth` - The annual population growth rate, as a percentage.
- `area` - The total land and water area.

In [32]:
import sqlalchemy

sqlalchemy.create_engine('sqlite:///factbook.db')
%load_ext sql
%sql sqlite:///factbook.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @factbook.db'

In [33]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


## Data overview

Let's display the first 5 rows of the table `facts` and get familiar with the data.

In [34]:
%%sql
SELECT * 
  FROM facts 
 LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

- `name` - The name of the country.
- `area` - The country's total area (both land and water).
- `area_land` - The country's land area in square kilometers.
- `area_water` - The country's waterarea in square kilometers.
- `population` - The country's population.
- `population_growth` - The country's population growth as a percentage.
- `birth_rate` - The country's birth rate, or the number of births a year per 1,000 people.
- `death_rate` - The country's death rate, or the number of death a year per 1,000 people.

Let's start by calculating some summary statistics and look for any outlier countries.

## Exploring Outliers

First, we will take a look at the maximum and minimum values for `population` and `population_growth`. 

In [35]:
%%sql 
SELECT MIN(population), MAX(population), 
       MIN(population_growth), MAX(population_growth)
  FROM facts;


 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


A few things stick out from the summary statistics in the last screen:

- There's a country with a population of 0
- There's a country with a population of 7256490011 (or more than 7.2 billion people)

Let's use subqueries to zoom in on just these countries without using the specific values.

In [36]:
%%sql 
SELECT name, MAX(population)
  FROM facts;

 * sqlite:///factbook.db
Done.


name,MAX(population)
World,7256490011


In [37]:
%%sql 
SELECT name, MIN(population)
  FROM facts;

 * sqlite:///factbook.db
Done.


name,MIN(population)
Antarctica,0


It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion. It also seems like the table contains a row for Antarctica, which explains the population of 0.

Now that we know this, we should recalculate the summary statistics we calculated earlier, while excluding the row for the whole world.

## Exploring Average Population and Area

In  this step we will calculate the summary statistics excluding `World`.

In [38]:
%%sql 
SELECT MIN(population), MAX(population), 
       MIN(population_growth), MAX(population_growth)
  FROM facts
 WHERE name != 'World';

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


In [39]:
%%sql 
SELECT AVG(population), AVG(area)
  FROM facts
 WHERE name != 'World';

 * sqlite:///factbook.db
Done.


AVG(population),AVG(area)
32242666.56846473,555093.546184739


## Finding Densely Populated Countries

To finish, we'll build on the query we wrote for the previous step to find countries that are densely populated. We'll identify countries that have:

 - Above average values for population.
 - Below average values for area.

In [40]:
%%sql 
SELECT name, population, area
  FROM facts
 WHERE name != 'World'
   AND population > (SELECT AVG(population)
                       FROM facts)
   AND area < (SELECT AVG(area)
                 FROM facts);

 * sqlite:///factbook.db
Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Japan,126919659,377915
Philippines,100998376,300000
Thailand,67976405,513120
United Kingdom,64088222,243610
Vietnam,94348835,331210


## Finding most populated country 

In this step we will answer the following questions: 

- What country has the most people? 
- What country has the highest growth rate?


In [41]:
%%sql 
SELECT name, MAX(population)
  FROM facts
 WHERE name != 'World';

 * sqlite:///factbook.db
Done.


name,MAX(population)
China,1367485388


As expected, China is the most populated country in the world.

In [42]:
%%sql 
SELECT name, MAX(population_growth)
 FROM facts
 WHERE name != 'World';

 * sqlite:///factbook.db
Done.


name,MAX(population_growth)
South Sudan,4.02


South Sudan has the highest growth rate. 

## Exploring water to land ratios 

We will find out:

- which countries have the highest ratios of water to land.
- which countries have more water than land. 

In order to  compute the ratios, we will divide the `area_water` by `area_land`.

In [43]:
%%sql 
SELECT name, area_water, area_land, 
       area_water/area_land AS water_land_ratio
  FROM facts
ORDER BY water_land_ratio DESC;

 * sqlite:///factbook.db
Done.


name,area_water,area_land,water_land_ratio
British Indian Ocean Territory,54340,60,905
Virgin Islands,1564,346,4
Afghanistan,0,652230,0
Albania,1350,27398,0
Algeria,0,2381741,0
Andorra,0,468,0
Angola,0,1246700,0
Antigua and Barbuda,0,442,0
Argentina,43710,2736690,0
Armenia,1540,28203,0


In [44]:
%%sql 
SELECT name, area_water, area_land
  FROM facts 
 WHERE area_water > area_land;

 * sqlite:///factbook.db
Done.


name,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


As we can see, British Indian Ocean Territory has more water than land along with Virgin Islands as well as the highest water to land ratio. 

## Exploring birth and migration rates

To answer the question which countries will add the most people to their population next year, we need to find countries with the highest value of both birth rate and migration rate. 

In [45]:
%%sql 
SELECT name, ROUND(birth_rate + migration_rate) AS most_people_added
  FROM facts
ORDER BY most_people_added DESC;

 * sqlite:///factbook.db
Done.


name,most_people_added
Somalia,49.0
South Sudan,48.0
Mali,47.0
Niger,46.0
Uganda,45.0
American Samoa,44.0
Sao Tome and Principe,43.0
Zambia,43.0
Burkina Faso,42.0
Burundi,42.0


As we can see, Somalia, South Sudan and Mali are most likely to add more people to its population. 

## Death Rate VS Birth Rate

In this step we will identify which countries have a higher death rate than birth rate. 

In [46]:
%%sql 
SELECT name, birth_rate, death_rate
  FROM facts
 WHERE death_rate > birth_rate;

 * sqlite:///factbook.db
Done.


name,birth_rate,death_rate
Austria,9.41,9.42
Belarus,10.7,13.36
Bosnia and Herzegovina,8.87,9.75
Bulgaria,8.92,14.44
Croatia,9.45,12.18
Czech Republic,9.63,10.34
Estonia,10.51,12.4
Germany,8.47,11.42
Greece,8.66,11.09
Hungary,9.16,12.73


The table above represents countires where death rate is higher than birth rate. 

## Exploring Population/Area Ratio

What countries have the highest population/area ratio and how does it compare to list of densely populated counties from the previous step? Let's find out.

In [47]:
%%sql 
SELECT name, population/area AS population_area_ratio
  FROM facts 
ORDER BY population_area_ratio DESC;

 * sqlite:///factbook.db
Done.


name,population_area_ratio
Macau,21168
Monaco,15267
Singapore,8141
Hong Kong,6445
Gaza Strip,5191
Gibraltar,4876
Bahrain,1771
Maldives,1319
Malta,1310
Bermuda,1299



Countries with the highest population to area ratio are Macau, Monaco and Singapore. Whereas, countries with population higher than the average population, and the area smaller than the average area are Bangladesh, Germany, and Japan. The results  are different for obvious reasons, in the first case we compare the ratios of individual countries between each other, and in the second case, we compare each country's population and area values against the average of all countries. 

# Conclusion 

In this project, we analyzed the CIA Factbook Data containing the demographics of all countries in the world. We explored countries with highest/lowest population and population growth, found densely populated countries, calculated the water/land ratio and explored countries that have more water than land. We used SQL queries to accomplish all tasks including aggregate functions, 'group by' clause to compute summary statistics by unique value, nested subqueries and others. 